## Feature Extraction
#### 1)Import video file paths
#### 2) Rename video files to their base paths names
#### 3) Import ranamed video Files
#### 4) Extract Features from videos

## Renaming our Video files
###### 1) Importing video file Paths
###### 2) Renaming video files

In [ ]:
import os

def rename_files():

  main_dir = "Your Local disk File path"
  directories = os.listdir(main_dir)

  for directory in directories:
    i = 1;

    videos = os.listdir(f"{main_dir}/{directory}")
    for video in videos:
      video_initial_name = f"{main_dir}/{directory}/{video}"

      video_change_name = f"{main_dir}/{directory}/{directory}_{i}.mp4"
      os.rename(video_initial_name , video_change_name)
      print(f"File renamed from {video_initial_name} to {video_change_name}")
      i += 1;

In [ ]:
rename_files()


## Extracting Features From Videos

In [ ]:
!pip install mediapipe

In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import math
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import os

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


## Functions to calculate Angles and Distances

In [ ]:
def calculate_distance(point1, point2):
    return math.sqrt(
        (point2.x - point1.x) ** 2
        + (point2.y - point1.y) ** 2
        + (point2.z - point1.z) ** 2
    )


def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2( a[1] - b[1], a[0] - b[0] )

    angle = np.abs(radians * 180.0 / np.pi)

    return angle


## Function to Return Features from each video Frame

In [ ]:
 def get_features(landmarks):
            ## left body parts values

            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
            left_toe = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]


            ## Right body part values
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            right_toe = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]



            ## COORDS of left body parts
            left_shoulder_coords = [left_shoulder.x, left_shoulder.y, left_shoulder.z]
            left_elbow_coords = [left_elbow.x, left_elbow.y, left_elbow.z]
            left_wrist_coords = [left_wrist.x, left_wrist.y, left_wrist.z]
            left_hip_coords = [left_hip.x, left_hip.y, left_hip.z]
            left_knee_coords = [left_knee.x, left_knee.y, left_knee.z]
            left_ankle_coords = [left_ankle.x , left_ankle.y , left_ankle.z]

            ## COORDS OF Right Body Parts
            right_shoulder_coords = [ right_shoulder.x , right_shoulder.y , right_shoulder.z ]
            right_elbow_coords = [right_elbow.x, right_elbow.y, right_elbow.z]
            right_wrist_coords = [right_wrist.x, right_wrist.y, right_wrist.z]
            right_hip_coords = [right_hip.x, right_hip.y, right_hip.z]
            right_knee_coords = [right_knee.x , right_knee.y, right_knee.z]
            right_ankle_coords = [right_ankle.x , right_ankle.y , right_ankle.z]

            ## CALCULATING DISTANCES
            left_hip_sh_dist = calculate_distance(left_hip, left_shoulder)
            right_hip_sh_dist = calculate_distance(right_hip, right_shoulder)

            left_wrist_sh_dist = calculate_distance(left_wrist, left_shoulder)
            right_wrist_sh_dist = calculate_distance(right_wrist, right_shoulder)

            left_ch_elbow_dist = calculate_distance(left_shoulder, left_elbow)
            right_ch_elbow_dist = calculate_distance(right_shoulder, right_elbow)

            left_wrist_hip_dist = calculate_distance(left_hip, left_wrist)
            right_wrist_hip_dist = calculate_distance(right_hip, right_wrist)

            left_elbow_wrist_dist = calculate_distance(left_elbow, left_wrist)
            right_elbow_wrist_dist = calculate_distance(right_elbow, right_wrist)

            left_vert_dist_sh_hip = left_shoulder.y - left_hip.y
            right_vert_dist_sh_hip = right_shoulder.y - right_hip.y

            left_horiz_dist_sh_hip = left_shoulder.x - left_hip.x
            right_horiz_dist_sh_hip = right_shoulder.x - right_hip.x

            left_wrist_height = left_shoulder.y - left_wrist.y
            right_wrist_height = right_shoulder.y - right_wrist.y


            ## CALCULATING ANGLES
            left_elbow_angle = calculate_angle(
                left_shoulder_coords, left_elbow_coords, left_wrist_coords
            )
            right_elbow_angle = calculate_angle(
                right_shoulder_coords, right_elbow_coords, right_wrist_coords
            )

            left_hip_angle = calculate_angle(
                left_shoulder_coords, left_hip_coords, left_knee_coords
            )
            right_hip_angle = calculate_angle(
                right_shoulder_coords, right_hip_coords, right_knee_coords
            )
            left_shoulder_flexion_extension_angle = calculate_angle(
                left_hip_coords, left_shoulder_coords, left_elbow_coords
            )
            right_shoulder_flexion_extension_angle = calculate_angle(
                right_hip_coords, right_shoulder_coords, right_elbow_coords
            )

            # Shoulder Abduction/Adduction
            left_shoulder_abduction_adduction_angle = calculate_angle(
                left_shoulder_coords, left_hip_coords, left_knee_coords
            )
            right_shoulder_abduction_adduction_angle = calculate_angle(
                right_shoulder_coords, right_hip_coords, right_knee_coords
            )

            # Body Tilt Angle (Tilt of the body relative to vertical axis)
            left_body_tilt_angle = calculate_angle(
                [left_shoulder.x, left_shoulder.y], [left_hip.x, left_hip.y], [0, 1]
            )
            right_body_tilt_angle = calculate_angle(
                [right_shoulder.x, right_shoulder.y], [right_hip.x, right_hip.y], [0, 1]
            )

            ## Ankle to Shoulder angle
            left_ankle_shoulder_angle = calculate_angle(
                left_shoulder_coords, left_hip_coords, left_ankle_coords
            )
            right_ankle_shoulder_angle = calculate_angle(
                right_shoulder_coords, right_hip_coords, right_ankle_coords
            )

            ## Ankle Shoulder Distance
            left_ankle_shoulder_distance = calculate_distance(
                left_ankle , left_shoulder
            )
            right_ankle_shoulder_distance = calculate_distance(
                right_ankle , right_shoulder
            )

            ## Ankle knee angle
            left_ankle_knee_angle = calculate_angle(
                left_knee_coords, left_hip_coords, left_ankle_coords
            )
            right_ankle_knee_angle = calculate_angle(
                right_knee_coords, right_hip_coords, right_ankle_coords
            )

            ## Elbow flexion angle
            left_elbow_flexion_angle = calculate_angle(left_shoulder_coords , left_wrist_coords , left_elbow_coords)
            right_elbow_flexion_angle = calculate_angle(right_shoulder_coords , right_wrist_coords , right_elbow_coords)

            # Knee flexion angle
            left_knee_flexion = calculate_angle(left_hip_coords , left_knee_coords , left_ankle_coords)
            right_knee_flexion = calculate_angle(right_hip_coords , right_knee_coords , right_ankle_coords)

            # Shoulder to knee distance
            left_shoulder_knee_distance = calculate_distance(left_shoulder , left_knee)
            right_shoulder_knee_distance = calculate_distance(right_shoulder , right_knee)

            # Hip to ankle distance
            left_hip_ankle_distance = calculate_distance(left_hip , left_ankle)
            right_hip_ankle_distance = calculate_distance(right_hip , right_ankle)

            # ankle dorsiflexion angle
            left_ankle_dorsiflexion_angle = calculate_angle( left_knee_coords , left_ankle_coords , left_toe)
            right_ankle_dorsiflexion_angle = calculate_angle( right_knee_coords , right_ankle_coords , right_toe )


            return (
                right_ankle_dorsiflexion_angle,
                left_ankle_dorsiflexion_angle,
                left_shoulder_knee_distance,
                right_shoulder_knee_distance,
                left_hip_ankle_distance,
                right_hip_ankle_distance,
                right_elbow_flexion_angle,
                left_elbow_flexion_angle,
                left_ankle_knee_angle,
                right_ankle_shoulder_distance,
                right_ankle_knee_angle,
                left_ankle_shoulder_distance,
                right_ankle_shoulder_angle,
                left_ankle_shoulder_angle,
                left_elbow_angle,
                right_elbow_angle,
                left_hip_angle,
                right_hip_angle,
                left_shoulder_flexion_extension_angle,
                right_shoulder_flexion_extension_angle,
                left_shoulder_abduction_adduction_angle,
                right_shoulder_abduction_adduction_angle,
                left_body_tilt_angle,
                right_body_tilt_angle,
                left_hip_sh_dist,
                right_hip_sh_dist,
                left_wrist_sh_dist,
                right_wrist_sh_dist,
                left_ch_elbow_dist,
                right_ch_elbow_dist,
                left_wrist_hip_dist,
                right_wrist_hip_dist,
                left_elbow_wrist_dist,
                right_elbow_wrist_dist,
                left_vert_dist_sh_hip,
                right_vert_dist_sh_hip,
                left_horiz_dist_sh_hip,
                right_horiz_dist_sh_hip,
                left_wrist_height,
                right_wrist_height,
            )

## Function to Read Video file and extract feature from each video

In [ ]:
def get_features_from_video(main_directory_path , excercise_name):

  directory = f"{main_directory_path}/{excercise_name}"
  video_paths = os.listdir(directory)

  extracted_feature_dir = f"/content/drive/MyDrive/New Extracted Features/{excercise_name}"

  if not os.path.exists(extracted_feature_dir):
    print(f"Creating Directory: {extracted_feature_dir}")
    os.makedirs(extracted_feature_dir)


  for video_path in video_paths:
    video_full_path = f"{directory}/{video_path}"
    print(video_full_path)
    cap = cv2.VideoCapture(video_full_path)
    base_name = os.path.basename(video_path)
    split_base_name = base_name.split('.')[0]
    print(f"Etracting Features for: {split_base_name}")


    features_list = []
    frame_number = 0

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
      while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print('leaving')
            break


        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # print(results)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            (
                right_ankle_dorsiflexion_angle,
                left_ankle_dorsiflexion_angle,
                left_shoulder_knee_distance,
                right_shoulder_knee_distance,
                left_hip_ankle_distance,
                right_hip_ankle_distance,
                right_elbow_flexion_angle,
                left_elbow_flexion_angle,
                left_ankle_knee_angle,
                right_ankle_shoulder_distance,
                right_ankle_knee_angle,
                left_ankle_shoulder_distance,
                right_ankle_shoulder_angle,
                left_ankle_shoulder_angle,
                left_elbow_angle,
                right_elbow_angle,
                left_hip_angle,
                right_hip_angle,
                left_shoulder_flexion_extension_angle,
                right_shoulder_flexion_extension_angle,
                left_shoulder_abduction_adduction_angle,
                right_shoulder_abduction_adduction_angle,
                left_body_tilt_angle,
                right_body_tilt_angle,
                left_hip_sh_dist,
                right_hip_sh_dist,
                left_wrist_sh_dist,
                right_wrist_sh_dist,
                left_ch_elbow_dist,
                right_ch_elbow_dist,
                left_wrist_hip_dist,
                right_wrist_hip_dist,
                left_elbow_wrist_dist,
                right_elbow_wrist_dist,
                left_vert_dist_sh_hip,
                right_vert_dist_sh_hip,
                left_horiz_dist_sh_hip,
                right_horiz_dist_sh_hip,
                left_wrist_height,
                right_wrist_height,
            ) = get_features(landmarks)

            features_list.append(
                [
                right_ankle_dorsiflexion_angle,
                left_ankle_dorsiflexion_angle,
                left_shoulder_knee_distance,
                right_shoulder_knee_distance,
                left_hip_ankle_distance,
                right_hip_ankle_distance,
                right_elbow_flexion_angle,
                left_elbow_flexion_angle,
                left_ankle_knee_angle,
                right_ankle_shoulder_distance,
                right_ankle_knee_angle,
                left_ankle_shoulder_distance,
                right_ankle_shoulder_angle,
                left_ankle_shoulder_angle,
                left_elbow_angle,
                right_elbow_angle,
                left_hip_angle,
                right_hip_angle,
                left_shoulder_flexion_extension_angle,
                right_shoulder_flexion_extension_angle,
                left_shoulder_abduction_adduction_angle,
                right_shoulder_abduction_adduction_angle,
                left_body_tilt_angle,
                right_body_tilt_angle,
                left_hip_sh_dist,
                right_hip_sh_dist,
                left_wrist_sh_dist,
                right_wrist_sh_dist,
                left_ch_elbow_dist,
                right_ch_elbow_dist,
                left_wrist_hip_dist,
                right_wrist_hip_dist,
                left_elbow_wrist_dist,
                right_elbow_wrist_dist,
                left_vert_dist_sh_hip,
                right_vert_dist_sh_hip,
                left_horiz_dist_sh_hip,
                right_horiz_dist_sh_hip,
                left_wrist_height,
                right_wrist_height,
                ]
            )

      df = pd.DataFrame(features_list, columns=[
                                                "right_ankle_dorsiflexion_angle",
                                                "left_ankle_dorsiflexion_angle",
                                                "left_shoulder_knee_distance",
                                                "right_shoulder_knee_distance",
                                                "left_hip_ankle_distance",
                                                "right_hip_ankle_distance",
                                                "right_elbow_flexion_angle",
                                                "left_elbow_flexion_angle",
                                                "left_ankle_knee_angle",
                                                "right_ankle_shoulder_distance",
                                                "right_ankle_knee_angle",
                                                "left_ankle_shoulder_distance",
                                                "right_ankle_shoulder_angle",
                                                "left_ankle_shoulder_angle",
                                                "left_elbow_angle",
                                                "right_elbow_angle",
                                                "left_hip_angle",
                                                "right_hip_angle",
                                                "left_shoulder_flexion_extension_angle",
                                                "right_shoulder_flexion_extension_angle",
                                                "left_shoulder_abduction_adduction_angle",
                                                "right_shoulder_abduction_adduction_angle",
                                                "left_body_tilt_angle",
                                                "right_body_tilt_angle",
                                                "left_hip_sh_dist",
                                                "right_hip_sh_dist",
                                                "left_wrist_sh_dist",
                                                "right_wrist_sh_dist",
                                                "left_ch_elbow_dist",
                                                "right_ch_elbow_dist",
                                                "left_wrist_hip_dist",
                                                "right_wrist_hip_dist",
                                                "left_elbow_wrist_dist",
                                                "right_elbow_wrist_dist",
                                                "left_vert_dist_sh_hip",
                                                "right_vert_dist_sh_hip",
                                                "left_horiz_dist_sh_hip",
                                                "right_horiz_dist_sh_hip",
                                                "left_wrist_height",
                                                "right_wrist_height"
                                                ]

                                             )
    if not os.path.exists(f'{extracted_feature_dir}/{split_base_name}.csv'):
      print(f"Writing Features of {split_base_name} to csv")
      df.to_csv(f'{extracted_feature_dir}/{split_base_name}.csv', index=False)
    cap.release()



In [ ]:
main_dir = "Your Data Location"
directories = os.listdir(main_dir) # ['bench press', 'push-up', 'bicep curl', 'rest']

for directory in directories:
  print(directory)
  get_features_from_video(main_dir , directory)


## Setting Up our Model
1.   Importing Libraries
2.   Importing Main Workout Folder i.e Bench Press , Push-up , Bicep Curl etc
3.   Creating Sequences
4.   



## Libraries for Model Training

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import os


## Importing Workout Folders

In [ ]:
folder_directory = "/content/drive/MyDrive/New Extracted Features"
# os.listdir(folder_directory)
folder_names = os.listdir(folder_directory)
folder_names

labels = folder_names
labels

## Creating Sequences

In [ ]:
SEQ_LENGTH = 40 ## Set a Sequences length of your choice
LOOKBACK = 15   ## Look Back includes the last n frames to the next sequence

def generate_sequences_with_lookback_csv(csv_path, seq_length, lookback, label):
    df = pd.read_csv(csv_path)
    features = df.values
    print(len(features))

    sequences = []
    labels = []

    for i in range(0, len(features), seq_length - lookback):
      ## @TODO use a better way
      if i+seq_length>len(features):
        break
      sequence = features[i:i+seq_length]
      sequences.append(sequence)
      labels.append(label)
    return (sequences,labels)

In [ ]:
X = [] ## Contains Sequences of All Features
Y = []  ## Contains Seuquences of All Labels. each sequence has a single Label


for label in labels:
  print(f"Processing {label}")
  folder_path = f"{folder_directory}/{label}"
  folders = [f"{folder_path}/{file}" for file in os.listdir(folder_path)]
  # print(folders)

  for file in folders:
    print(f"Processing { file }")
    sequences,results = generate_sequences_with_lookback_csv(file , SEQ_LENGTH , LOOKBACK ,label)
    X.extend(sequences)
    Y.extend(results)



###### Checking shapes x.shape = (n , m , x) , n = total sequences , m = sequence length , x = number of Features

###### Checking shape y.shape = (n) , n = number of labels,
###### **NOTE: both n of X and Y should be same ** **bold text**

In [ ]:
np.array(X).shape

In [ ]:
np.array(Y).shape

###### Encoding Labels

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(Y)
y_encoded

In [ ]:
print(np.unique(y_encoded)  , np.unique(Y)) ## This will give the encoded number responding to its label

###### Converting Encoded Labels to Catgories of len(np.unique(Y))

In [ ]:
from tensorflow.keras.utils import to_categorical
# One-hot encode Y_train
label_categorical = to_categorical(y_encoded, num_classes= len(np.unique(Y)))
label_categorical

In [ ]:
np.shape(label_categorical) ## This should have the same value as the original Y

In [ ]:
classes =  label_encoder.classes_
classes

## Normalizing X


In [ ]:
X = np.array(X)
scaler = MinMaxScaler()
num_samples, sequence_length, num_features = X.shape

X_reshaped = X.reshape(-1, num_features)

# # Fit and transform
X_normalized = scaler.fit_transform(X_reshaped)

# # Reshape back to original 3D shape
X_normalized = X_normalized.reshape(num_samples, sequence_length, num_features)

print(X_normalized.shape)


## Splitting into Train Test Split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_normalized, label_categorical , test_size=0.2, random_state=42)

In [ ]:
X_train.shape , Y_train.shape,  X_test.shape ,Y_test.shape

## Setting Up our Model
#### Using Double LSTM architecture

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LayerNormalization

model = Sequential()

# First LSTM layer converted to Bidirectional
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(sequence_length, num_features), recurrent_dropout= 0.3))
model.add(LayerNormalization(axis=1))

# Second LSTM layer converted to Bidirectional
model.add(LSTM(64, return_sequences = False, activation='relu' ,  recurrent_dropout= 0.3))
model.add(LayerNormalization(axis=1))


model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense( len(np.unique(Y)) , activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


#### Using Checkpointing to save Model at regular intervals
#### Using Early Stopping to prevent overfitting

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filepath= f"/content/drive/MyDrive/New Models/Model-CNN-LSTM-seq-{SEQ_LENGTH}-LB-{LOOKBACK}.keras",
    # save_weights_only=True,  # Saves only the model's weights
    monitor='val_loss',      # Monitor the validation loss
    mode='min',              # Mode to select the best model (minimizing val_loss)
    save_best_only=True,     # Save only the best model based on the monitored metric
    verbose=1                # Print a message when saving the model
  )

early_stopping = EarlyStopping(monitor='val_loss', patience=10 , restore_best_weights=True)

#### Fitting Our Model

In [ ]:
model.fit(X_train, Y_train,
          epochs=100, batch_size=32,
          validation_data=(X_test, Y_test),
          callbacks=[checkpoint_callback , early_stopping],
          verbose=1)  # Set verbose=1 to see the training progress